In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, MultiLineString
import re
import geopandas as gpd
from shapely.geometry import Point
from shapely.strtree import STRtree
from collections import defaultdict
import geopandas as gpd
from shapely import shortest_line
# Load your lines


wärme  = gpd.read_file(r"C:\dev\ews\lk\all\data\lk_waerme.gpkg")


In [2]:
output = gpd.GeoDataFrame(columns=["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "ID", "GEBID", "eigentuemer", "status", "distance", "Koordinaten", "Lokationsart" ,"geometry"])

In [3]:
wärme = wärme.drop(columns=['fileid', 'obj_id','medium', 'datenherr','datenlieferant','letzte_aenderung','lagebestimmung','breite','objektart','letzte_lieferung'])

In [4]:
Adressen = gpd.read_file("C:\dev\ews\Adressen_GIS-ZH_-OGD/Adressen_GIS-ZH_-OGD.gpkg")

gebäude = gpd.read_file("C:\dev\ews\AV_MOpublic-_Bodenbedeckung_-OGD\AV_MOpublic-_Bodenbedeckung_-OGD.gpkg", layer="AVZH_BODENBEDECKUNG_F")

proj_gebäude = gpd.read_file("C:\dev\ews\AV_MOpublic-_Bodenbedeckung_-OGD\AV_MOpublic-_Bodenbedeckung_-OGD.gpkg", layer="AVZH_BODENBEDECKUNG_PROJ_F")

liegeschaften = gpd.read_file("C:\dev\ews\AV_MOpublic-_Liegenschaften_-OGD\AV_MOpublic-_Liegenschaften_-OGD.gpkg", layer="AVZH_LIEGENSCHAFTEN_F")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\rigli\AppData\Local\Temp\ipykernel_22944\548821143.py:1: SyntaxWarning: invalid escape sequence '\d'
  Adressen = gpd.read_file("C:\dev\ews\Adressen_GIS-ZH_-OGD/Adressen_GIS-ZH_-OGD.gpkg")
C:\Users\rigli\AppData\Local\Temp\ipykernel_22944\548821143.py:3: SyntaxWarning: invalid escape sequence '\d'
  gebäude = gpd.read_file("C:\dev\ews\AV_MOpublic-_Bodenbedeckung_-OGD\AV_MOpublic-_Bodenbedeckung_-OGD.gpkg", layer="AVZH_BODENBEDECKUNG_F")
C:\Users\rigli\AppData\Local\Temp\ipykernel_22944\548821143.py:5: SyntaxWarning: invalid escape sequence '\d'
  proj_gebäude = gpd.read_file("C:\dev\e

In [5]:
#remove useless attributes
liegeschaften.drop(columns=['OBJID','NBIDENT','NUMMER','VOLLSTAENDIGKEIT','FLAECHENMASS','BFSNR','BEARBEITUNGSDATUM','SHAPE.AREA','SHAPE.LEN'], inplace=True)


#filter only gebäude, remove useless attributes
gebäude = gebäude[(gebäude['ART'] == 'Gebäude')]
proj_gebäude = proj_gebäude[proj_gebäude['ARTZHID'].isin([0, 1, 2, 3, 4, 5, 6, 7])]

gebäude.drop(columns=['OBJID','QUALITAET','ART','BEARBEITUNGSDATUM','ARTCHID','ARTZHID','GVZNUMMER','SHAPE.AREA','SHAPE.LEN'], inplace=True)
proj_gebäude.drop(columns=['OBJID','QUALITAET','ART','BEARBEITUNGSDATUM','ARTCHID','ARTZHID','GVZNUMMER','SHAPE.AREA','SHAPE.LEN','BEWILLIGUNGSSTATUS'], inplace=True)


In [6]:
gebäude['Projektiert'] = 'Nein'
proj_gebäude['Projektiert'] = 'Ja'

In [7]:
all_gebäude = pd.concat([gebäude, proj_gebäude])

all_gebäude =gpd.GeoDataFrame(all_gebäude, geometry='geometry')

all_gebäude['GEBID'] = range(len(all_gebäude))

all_gebäude["Koordinaten"] = all_gebäude.geometry.apply(
    lambda geom: f"{round(geom.exterior.coords[0][0])} {round(geom.exterior.coords[0][1])}")

In [8]:

# Create GeoDataFrame for first vertices
first_vertices = gpd.GeoDataFrame(
    wärme.drop(columns='geometry'),  # keep other columns if needed
    geometry=wärme.geometry.apply(lambda line: Point(line.coords[0])),
    crs=wärme.crs
)

# Create GeoDataFrame for last vertices
last_vertices = gpd.GeoDataFrame(
    wärme.drop(columns='geometry'),  # keep other columns if needed
    geometry=wärme.geometry.apply(lambda line: Point(line.coords[-1])),
    crs=wärme.crs
)

# Combine into one GeoDataFrame if desired
all_vertices = gpd.GeoDataFrame(
    pd.concat([first_vertices, last_vertices], ignore_index=True),
    crs=wärme.crs
)



In [9]:
# Drop duplicate geometries
unique_vertices = all_vertices.drop_duplicates(subset='geometry')

In [10]:
points = unique_vertices

# Buffer points by 5 cm (0.05 meters)
points['buffered'] = points.geometry.buffer(0.05)

# Create a GeoDataFrame from buffered geometries for spatial join
buffer_gdf = gpd.GeoDataFrame(points.drop(columns='geometry'), geometry=points['buffered'], crs=points.crs)

# Spatial join: find which buffered points intersect which lines
join = gpd.sjoin(buffer_gdf, wärme, how='left', predicate='intersects')

# Count number of unique lines each buffered point intersects
counts = join.groupby(join.index).size()

# Filter points that intersect less than 2 lines (keep those)
keep_points = counts[counts < 2].index

# Keep original points corresponding to these indices
anschlüsse = points.loc[keep_points]

print(f"Original points count: {len(points)}")
print(f"Filtered points count (intersect < 2 lines): {len(anschlüsse)}")

c:\dev\GWR\.venv\Lib\site-packages\geopandas\geodataframe.py:1969: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Original points count: 54708
Filtered points count (intersect < 2 lines): 23327


In [11]:
# Drop the extra geometry column ('buffered') before saving
anschlüsse = anschlüsse.drop(columns='buffered', errors='ignore')
# Make sure the active geometry is 'geometry' column
anschlüsse.set_geometry('geometry', inplace=True)
anschlüsse['ID'] = range(len(anschlüsse))
anschlüsse.to_file("wärme_output.gpkg", layer='WÄRMEANSCHLUSS_P')

In [12]:
Adressen['Adresse'] = Adressen['STRASSENNAME'] + ' ' + Adressen['HAUSNUMMER']
Adressen_clean = Adressen[Adressen['GWR_EGID'].notna()]
Adressen_subset = Adressen_clean[['Adresse','GWR_EGID','PLZ','ORTSCHAFTSNAME']]

In [ ]:
gebäude_Adressen = all_gebäude.merge(Adressen_subset, how='left', on='GWR_EGID')

,GWR_EGID,BFSNR,ARTZH,geometry,Projektiert,GEBID,Koordinaten,Adresse,PLZ,ORTSCHAFTSNAME
0,33318.0,91,Gebäude Landwirtschaft,"POLYGON ((2669654.39 1261177.714, 2669658.36 1...",Nein,0,2669654 1261178,Surbengraben 1,8166.0,Niederweningen
1,210236081.0,91,Gebäude Landwirtschaft,"POLYGON ((2669750.389 1259897.149, 2669752.265...",Nein,1,2669750 1259897,Steinhof 1.1,8166.0,Niederweningen
2,210236084.0,91,Gebäude Landwirtschaft,"POLYGON ((2670034.659 1261308.388, 2670037.922...",Nein,2,2670035 1261308,Sandacher 1.3,8166.0,Niederweningen
3,210236074.0,91,Gebäude Landwirtschaft,"POLYGON ((2670064.04 1261284.304, 2670060.006 ...",Nein,3,2670064 1261284,Sandacher 1.2,8166.0,Niederweningen
4,33109.0,91,Gebäude Wohnen,"POLYGON ((2670096.231 1261845.328, 2670093.111...",Nein,4,2670096 1261845,Breitstrasse 1,8166.0,Niederweningen
...,...,...,...,...,...,...,...,...,...,...
359200,192058293.0,120,Gebäude Wohnen,"POLYGON ((2711319.308 1236775.877, 2711321.022...",Ja,346336,2711319 1236776,Birkenstrasse 4,8636.0,Wald ZH
359201,192026188.0,114,Gebäude Wohnen,"POLYGON ((2711377.1 1241441.993, 2711376.704 1...",Ja,346337,2711377 1241442,Rosenbergstrasse 110b,8498.0,Gibswil
359202,192024987.0,114,Nebengebäude,"POLYGON ((2712100.333 1242674.353, 2712098.76 ...",Ja,346338,2712100 1242674,Fistelstrasse 89.3,8497.0,Fischenthal
359203,NaN,120,Gebäude Wohnen,"POLYGON ((2712245.232 1240454.44, 2712237.768 ...",Ja,346339,2712245 1240454,NaN,NaN,NaN


In [28]:
filtered_gebäude = gebäude_Adressen[
    ~(
        (gebäude_Adressen["Adresse"].isna() &
         (gebäude_Adressen["ARTZH"] == "Nebengebäude")))
]


# filtered_gebäude = gebäude_Adressen[
#     ~(
#         (gebäude_Adressen["Adresse"].isna() &
#          (gebäude_Adressen["ARTZH"] == "Nebengebäude"))
#         |
#         (gebäude_Adressen["GWR_EGID"].isna())
#     )
# ]


In [29]:
def filter_out_number_dot_number(address):
    # Ensure it's a string before applying regex
    address = str(address)
    return not re.search(r'(?:\d+)?\.\d', address)
    
    
valid_gebäude = filtered_gebäude[filtered_gebäude['Adresse'].apply(filter_out_number_dot_number)]


valid_gebäude.to_file("wärme_output.gpkg", layer='ALLE_VALIDE_GEBÄUDE_F')
liegeschaften.to_file("wärme_output.gpkg", layer='LIEGENSCHAFTEN_F')

In [30]:
valid_gebäude

,GWR_EGID,BFSNR,ARTZH,geometry,Projektiert,GEBID,Koordinaten,Adresse,PLZ,ORTSCHAFTSNAME
0,33318.0,91,Gebäude Landwirtschaft,"POLYGON ((2669654.39 1261177.714, 2669658.36 1...",Nein,0,2669654 1261178,Surbengraben 1,8166.0,Niederweningen
4,33109.0,91,Gebäude Wohnen,"POLYGON ((2670096.231 1261845.328, 2670093.111...",Nein,4,2670096 1261845,Breitstrasse 1,8166.0,Niederweningen
5,33088.0,91,Gebäude Wohnen,"POLYGON ((2669928.461 1262382.101, 2669936.233...",Nein,5,2669928 1262382,Märzenbrunnen 1,8166.0,Niederweningen
6,210195948.0,91,Gebäude Wohnen,"POLYGON ((2669986.357 1262408.551, 2669986.155...",Nein,6,2669986 1262409,Haselstrasse 9,8166.0,Niederweningen
7,210215140.0,91,Gebäude Wohnen,"POLYGON ((2670037.308 1262458.57, 2670042.844 ...",Nein,7,2670037 1262459,Murzlenstrasse 62,8166.0,Niederweningen
...,...,...,...,...,...,...,...,...,...,...
359198,192064060.0,120,Gebäude Wohnen,"POLYGON ((2711185.124 1237580.643, 2711163.887...",Ja,346334,2711185 1237581,Binzholzstrasse 27,8636.0,Wald ZH
359200,192058293.0,120,Gebäude Wohnen,"POLYGON ((2711319.308 1236775.877, 2711321.022...",Ja,346336,2711319 1236776,Birkenstrasse 4,8636.0,Wald ZH
359201,192026188.0,114,Gebäude Wohnen,"POLYGON ((2711377.1 1241441.993, 2711376.704 1...",Ja,346337,2711377 1241442,Rosenbergstrasse 110b,8498.0,Gibswil
359203,NaN,120,Gebäude Wohnen,"POLYGON ((2712245.232 1240454.44, 2712237.768 ...",Ja,346339,2712245 1240454,NaN,NaN,NaN


In [47]:
#spatial join with gebäude
gebäude_anschluss = valid_gebäude.sjoin(anschlüsse, how='inner', predicate='intersects')

#define Lokationsart
gebäude_anschluss['Lokationsart'] = 'Anschluss in Gebäude'

output = gpd.GeoDataFrame(pd.concat([output, gebäude_anschluss[["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "ID", "GEBID", "eigentuemer", "status", "Koordinaten", "Lokationsart" ,"geometry"]]], ignore_index=True))


C:\Users\rigli\AppData\Local\Temp\ipykernel_17020\1169806202.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output = gpd.GeoDataFrame(pd.concat([output, gebäude_anschluss[["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "ID", "GEBID", "eigentuemer", "status", "Koordinaten", "Lokationsart" ,"geometry"]]], ignore_index=True))


In [48]:
gebäude_anschluss.to_file("wärme_output.gpkg", layer='ANSCHLUSS_IN_GEBÄUDE_F')

In [49]:
not_geb_joined = anschlüsse[~anschlüsse['ID'].isin(output['ID'])]
not_geb_joined_geb = valid_gebäude[~valid_gebäude['GEBID'].isin(output['GEBID'])]

In [50]:

# ─────────────────────────────────────────────────────────────
# Step 1: Spatial join to get parcel ID for points and buildings

anschluss_liegenschaft = gpd.sjoin(
    not_geb_joined, liegeschaften, how="left", predicate='within'
)

# Save original geometry and use centroids for spatial join
not_geb_joined_geb['orig_geom'] = not_geb_joined_geb.geometry
not_geb_joined_geb['geometry'] = not_geb_joined_geb['orig_geom'].centroid

gebäude_liegenschaft = gpd.sjoin(
    not_geb_joined_geb, liegeschaften, how="left", predicate='within'
)

# Restore original geometry
gebäude_liegenschaft['geometry'] = gebäude_liegenschaft['orig_geom']
gebäude_liegenschaft = gebäude_liegenschaft.drop(columns=['orig_geom'])


c:\dev\GWR\.venv\Lib\site-packages\geopandas\geodataframe.py:1969: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [51]:


# # Suppose ansch_gdf is your anschluss_liegenschaft GeoDataFrame
# # and buildings_gdf is gebäude_liegenschaft

# # Optionally, filter only matching parcels first (to reduce candidates)
# # For example, do a spatial join or filter buildings by parcel.

# def find_best_building_with_line(point_geom, parcel_id, buildings_gdf):
#     # Filter buildings in the same parcel (or pre-indexed)
#     candidates = buildings_gdf[buildings_gdf['EGRIS_EGRID'] == parcel_id]
#     if candidates.empty:
#         return None, None  # no match

#     best_dist = float('inf')
#     best_building = None
#     best_line = None

#     for _, b in candidates.iterrows():
#         b_geom = b.geometry
#         # compute shortest line between point and building
#         line = shortest_line(point_geom, b_geom)
#         d = line.length
#         if d < best_dist:
#             best_dist = d
#             best_line = line
#             best_building = b

#     return best_building, best_line

# # Now apply in a loop (or vectorized) to your anschluss_gdf
# results = []
# for _, a in anschluss_liegenschaft.iterrows():
#     p = a.geometry
#     pid = a['EGRIS_EGRID']
#     b_row, line = find_best_building_with_line(p, pid, gebäude_liegenschaft)
#     # Skip if no building found
#     if b_row is None:
#         continue

#     results.append({
#     'ansch_id': a.name,
#     'eigentuemer': a['eigentuemer'],
#     'status': a['status'],
#     'ID': a['ID'],
#     'GWR_EGID': b_row['GWR_EGID'],
#     'GEBID': b_row['GEBID'],
#     'line': line,
#     'distance': line.length if line is not None else None,
# })

# # Convert results to a GeoDataFrame if you like
# nearest_building_line = gpd.GeoDataFrame(results, geometry='line', crs=gebäude_liegenschaft.crs)


In [52]:
# nearest_building_line = nearest_building_line.dropna(subset=['distance'])


# nearest_building_line.to_file("wärme_output.gpkg", layer='NÄCHSTES_GEBÄUDE_IN_LIEGENSCHAFT_L')

In [53]:

# nearest_building_line_subset = nearest_building_line.drop(columns=['line','ansch_id'])

# nearest_building = (
#     gebäude_liegenschaft
#     .merge(nearest_building_line_subset, on="GEBID")
#     .rename(columns={"GWR_EGID_x": "GWR_EGID"})
#     .drop(columns=["GWR_EGID_y"], errors="ignore")
# )

# nearest_building['Lokationsart'] = 'Nächstes Gebäude in Liegenschaft'

# output = gpd.GeoDataFrame(pd.concat([output, nearest_building[["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "ID", "GEBID", "eigentuemer", "status", "Koordinaten", "distance", "Lokationsart" ,"geometry"]]], ignore_index=True))


# nearest_building.to_file("wärme_output.gpkg", layer='NÄCHSTES_GEBÄUDE_IN_LIEGENSCHAFT_F')

In [54]:
all_building_in_liegenschaft = anschluss_liegenschaft.merge(gebäude_liegenschaft, on='EGRIS_EGRID')

In [55]:
all_building_in_liegenschaft = all_building_in_liegenschaft.drop(columns=['geometry_x', 'index_right_x', 'index_right_y', 'EGRIS_EGRID'])

In [56]:
# Set the geometry column
all_building_in_liegenschaft = all_building_in_liegenschaft.set_geometry('geometry_y')

# Rename it to 'geometry' if it's not already called that
all_building_in_liegenschaft = all_building_in_liegenschaft.rename_geometry('geometry')

In [57]:
all_building_in_liegenschaft.to_file("wärme_output.gpkg",layer="ALLE_GEBÄUDE_IN_PARZELLE_F")

all_building_in_liegenschaft['Lokationsart'] = 'Alle Gebäude in der Parzelle'

output = gpd.GeoDataFrame(pd.concat([output, all_building_in_liegenschaft[["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "ID", "GEBID", "eigentuemer", "status", "Koordinaten", "Lokationsart" ,"geometry"]]], ignore_index=True))


In [58]:
not_parz_joined = anschlüsse[~anschlüsse['ID'].isin(output['ID'])]
not_parz_joined_geb = valid_gebäude[~valid_gebäude['GEBID'].isin(output['GEBID'])]

In [59]:

building_sindex = not_parz_joined_geb.sindex

def find_best_building_with_line(point_geom, buildings_gdf, sindex, search_radius=100):
    """
    Finds the closest building to a point using the shortest_line.
    Uses spatial index for fast lookup.
    
    Returns:
        - best matching building row (GeoSeries)
        - shortest line (LineString)
    """
    # Bounding box around the point
    bbox = point_geom.buffer(search_radius).bounds
    candidate_idx = list(sindex.intersection(bbox))

    if not candidate_idx:
        return None, None

    candidates = buildings_gdf.iloc[candidate_idx]

    best_dist = float('inf')
    best_building = None
    best_line = None

    for _, b in candidates.iterrows():
        b_geom = b.geometry
        line = shortest_line(point_geom, b_geom)
        d = line.length

        if d < best_dist:
            best_dist = d
            best_line = line
            best_building = b

    return best_building, best_line

# ------------------------------
# Apply to all points in anschluss_liegenschaft
# ------------------------------

results = []

for _, a in not_parz_joined.iterrows():
    p = a.geometry
    b_row, line = find_best_building_with_line(
        point_geom=p,
        buildings_gdf=not_parz_joined_geb,
        sindex=building_sindex,
        search_radius=100  # adjust as needed
    )

    # Skip if no building found
    if b_row is None:
        continue

    results.append({
    'ansch_id': a.name,
    'eigentuemer': a['eigentuemer'],
    'status': a['status'],
    'ID': a['ID'],
    'GWR_EGID': b_row['GWR_EGID'],
    'GEBID': b_row['GEBID'],
    'line': line,
    'distance': line.length if line is not None else None,
    })

# ------------------------------
# Convert to GeoDataFrame
# ------------------------------

nearest_building_line_all = gpd.GeoDataFrame(results, geometry='line', crs=gebäude_liegenschaft.crs)

In [ ]:
nearest_building_line_all = nearest_building_line_all[
    nearest_building_line_all['distance'].notna() &
    (nearest_building_line_all['distance'] < 10)
]


nearest_building_line_all.to_file("wärme_output.gpkg", layer='NÄCHSTES_GEBÄUDE_L')

nearest_building_line_all_subset = nearest_building_line_all.drop(columns=['line','ansch_id'])

nearest_building_all = (
    valid_gebäude
    .merge(nearest_building_line_all_subset, on="GEBID")
    .rename(columns={"GWR_EGID_x": "GWR_EGID"})
    .drop(columns=["GWR_EGID_y"], errors="ignore")
)

nearest_building_all['Lokationsart'] = 'Nächstes Gebäude in angrenzender Parzelle'

nearest_building_all.to_file("wärme_output.gpkg", layer='NÄCHSTES_GEBÄUDE_F')

output = gpd.GeoDataFrame(pd.concat([output, nearest_building_all[["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "ID", "GEBID", "eigentuemer", "status", "Koordinaten", "distance", "Lokationsart" ,"geometry"]]], ignore_index=True))


C:\Users\rigli\AppData\Local\Temp\ipykernel_17020\3491238079.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output = gpd.GeoDataFrame(pd.concat([output, nearest_building_all[["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "ID", "GEBID", "eigentuemer", "status", "Koordinaten", "distance", "Lokationsart" ,"geometry"]]], ignore_index=True))


In [ ]:
import datetime

today = datetime.date.today()


# output['GWR_EGID'] = output['GWR_EGID'].fillna(0).astype(int)
output = output.dropna(subset=['GWR_EGID'])
output['GWR_EGID'] = output['GWR_EGID'].astype(int)


output = output.rename(columns={"GWR_EGID": "EGID", "ORTSCHAFTSNAME": "Ort", "eigentuemer": "Netzbetreiber", "Koordinaten": "x/y-Koordinate", "status": "Leitungssatus", "distance": "Gebäudedistanz"})

output = output.drop("geometry", axis=1)

output.to_csv(f"wärme_output_{today}.csv")